# 카드 짝 맞추기


# 외벽 점검


In [ ]:
from itertools import permutations
from collections import deque
def solution(n, weak, dist):
    dist.sort(reverse = True)
    for i in range(1, len(dist)+1):
        ps = list(permutations(dist, i))
        for p in ps:
            for j in range(len(weak)):
                l = weak[:j]
                r = weak[j:]
                weak_list = deque(r + [x+n for x in l])
                p_ = deque(p[:])
                while weak_list and p_:
                    friend = p_.popleft()
                    weak_point = weak_list.popleft()
                    cover = weak_point + friend
                    while weak_list and cover >= weak_list[0]:
                        weak_list.popleft()
                if not weak_list:
                    return i
    return -1

# 접근법 참고해서 풀었음
# 완전탐색 + 원형탐색 + 순열 (졸라 복잡하네)
# 모든조합을 고려하되 탐색 완료되면 바로 리턴하는 식으로

# 좋아요 가장 많은 풀이
from collections import deque

def solution(n, weak, dist):
    dist.sort(reverse=True)
    q = deque([weak])
    visited = set()
    visited.add(tuple(weak))
    for i in range(len(dist)):
        d = dist[i]
        for _ in range(len(q)):
            current = q.popleft()
            for p in current:
                l = p
                r = (p + d) % n
                if l < r:
                    temp = tuple(filter(lambda x: x < l or x > r, current))
                else:
                    temp = tuple(filter(lambda x: x < l and x > r, current))

                if len(temp) == 0:
                    return (i + 1)
                elif temp not in visited:
                    visited.add(temp)
                    q.append(list(temp))
    return -1

# 다들 풀이가 제각각인거 같기도 하고

# 매칭 점수

In [ ]:
import re

class ftn:
    def __init__(self):
        self.idx = 0
        self.base_point = 0
        self.extra_point = 0
        self.link_point = 0
        self.match_point = 0
        self.url = None
        self.hrefs = []

def basepoints(word, page):
    texts, cnt = re.sub('[^a-zA-Z]',' ',page).split(), 0
    for text in texts:
        if word == text.lower(): cnt+=1
    return cnt

def extract(page):
    url = re.findall('<meta property="og:url" content="\S+"',page)[0][33:-1]
    hrefs = [href[9:-1] for href in re.findall('<a href="\S+"',page)]
    return url, hrefs

def solution(word, pages):
    word = word.lower()
    htmls = {}
    for i, page in enumerate(pages):
        url, hrefs = extract(page)
        basepoint = basepoints(word, page)
        html = ftn()
        html.idx = i
        html.base_point = basepoint
        html.extra_point = len(hrefs)
        html.url = url
        html.hrefs = hrefs
        htmls[url] = html
    for url, html in htmls.items():
        for href in html.hrefs:
            if href in htmls:
                htmls[href].link_point += html.base_point/html.extra_point
    answer = []
    for url, html in htmls.items():
        html.match_point = html.base_point + html.link_point
        answer.append((html.idx, html.match_point))
    answer.sort(key = lambda x: x[1], reverse = True)
    return answer[0][0]

# 혼란하다 혼란해
# 단순 구현이지만 정규표현식을 잘쓰는게 중요한거 같다
# 기본점수랑 url,href 추출하는데 여러 시행착오가 있었음


# 좋아요 가장 많은 풀이
def getScore(word, page):
    import re
    return re.sub('[^a-z]+', '.', page.lower()).split('.').count(word.lower())

def solution(word, pages):
    webpages = {}
    for i, page in enumerate(pages):
        pagetitle = page.split('<meta property=\"og:url\" content=\"')[1].split('\"')[0]
        links = []
        for link_long in page.split('a href=\"')[1:]:
            links.append(link_long.split('\"')[0])
        webpages[pagetitle] = [i, getScore(word, page), links, 0] #3은 링크점수

    for page in webpages.values():
        for target in page[2]:
            try:
                webpages[target][3] += page[1] / len(page[2])
            except:
                pass
    answer = []
    print(webpages)
    for page in webpages.values():
        answer.append([page[0], page[1] + page[3]])

    answer.sort(key=lambda x:x[0])
    return sorted(answer, key=lambda x:x[1], reverse=True)[0][0]

# 깔끔하다

import re

def solution(word, pages):
    score = []  # 인덱스(0), URL(1), 기본 점수(2), 링크(3), 링크 수(4)
    for j, k in enumerate(pages):
        k = k.replace('>','>\n')
        meta = re.findall('<meta.*', k[k.index('<head>')+6:k.index('</head>')])
        for i in meta:
            if re.findall('"https://.*"',i):
                url = re.findall('"https://.*"',i)[0][1:-1]
        link = [i[9:-1] for i in re.findall('<a href="https://.*"', k)]
        cnt = re.sub('[^a-zA-Z]', ' ', k).lower().split().count(word.lower())
        score.append([j, url, cnt, link, len(link)])

    res = []
    for x in score:
        link_score = sum([y[2] / y[4] for y in score if y[0] != x[0] and x[1] in y[3]])
        res.append([x[0], x[2] + link_score])
    res.sort(key=lambda x: x[1], reverse=True)

    return res[0][0]

# 깔끔하다2

# 110 옮기기

In [ ]:
def pick(s):
    cnt, stack = 0, []
    for s_ in s:
        if s_ == "0" and len(stack) > 1 and stack[-1] == stack[-2] == '1':
            stack.pop(); stack.pop();
            cnt += 1
        else:
            stack.append(s_)
    return ''.join(stack), cnt

def rejoin(s):
    for i in range(len(s)-1, -1, -1):
        if s[i] == '0':
            return s[:i+1] + '110' + s[i+1:]
    else:
        return '110' + s

def solution(s):
    answer = []
    for ss in s:
        new_s, cnt = pick(ss)
        for _ in range(cnt):
            new_s = rejoin(new_s)
        answer.append(new_s)
    return answer

# 110이 어디에 들어가느냐를 생각하는게 중요한거 같다
# 110이 존재하는 경우를 다 고려하고, 마지막부터 탐색해서 0 뒤에 110을 넣고,
# 만약 남은 문자열이 다 1이라면 그냥 맨앞에 두는게 사전순에서 가장 빠름
# 문자열 길이가 길다보니 deque를 쓸까했는데 머리에 박힌 구성이 stack이나 que나 다를게 없었음

# 다른 사람 풀이
def f(n):
    stk = []
    cnt = 0
    for i in n:
        stk.append(i)
        if i == '0' and stk[-3:] == ['1', '1', '0']:
            del stk[-3:]
            cnt += 1
    idx = -1
    for i in range(len(stk)):
        if stk[i] == '0':
            idx = i
    if idx < 0:
        ret = "110"*cnt + ''.join(stk)
    else:
        ret = ''.join(stk[:idx+1]) + "110"*cnt + ''.join(stk[idx+1:])
    return ret
def solution(s):
    return [f(x) for x in s]

# if i == '0' and stk[-3:] == ['1', '1', '0']:
#   del stk[-3:]
# -> pop을 여러번 안쓰고 이렇게 제거해도 되는군..
# ret = ''.join(stk[:idx+1]) + "110"*cnt + ''.join(stk[idx+1:])
# -> 0 뒤에만 들어가면 되니 110을 여러번 겹처서 return 하는거도 맞네

def solution(s):
    def f(x):
        left, I, IIO = '', 0, 0
        for i in x:
            if i == '1':
                I += 1
            elif I > 1:
                I -= 2
                IIO += 1
            else:
                left += '10' if I > 0 else '0'
                I = 0
        return left + '110' * IIO + '1' * I
    return [f(x) for x in s]

# 1을 남겨두고 110을 카운트해서 마지막에 return 하는건 똑같은데
# 중간 과정이 신박하네 / 뺸다라는 과정을 굳이 빼지않고 각 경우만 카운트하는 방법
# 실제로 시간도 훨씬 빠르다 / 배워두기

# 단속카메라


In [ ]:
def solution(routes):
    routes.sort(key = lambda x: (x[1],x[0]))
    tmp, cnt = -30001, 0
    for a, b in routes:
        if a > tmp:
            tmp = b
            cnt += 1
    return cnt

# 좋아요 가장 많은 풀이
def solution(routes):
    routes = sorted(routes, key=lambda x: x[1])
    last_camera = -30000
    answer = 0
    for route in routes:
        if last_camera < route[0]:
            answer += 1
            last_camera = route[1]
    return answer


# 블록 이동하기


In [ ]:
from collections import deque
def move(robot, board, N):
    (x1, y1), (x2, y2) = robot
    next_ = []
    if x1 == x2:
        if 0 <= y2 + 1 <= N - 1 and board[x2][y2 + 1] == 0:
            next_.append(((x1, y2), (x2, y2 + 1)))
        if 0 <= y1 - 1 <= N - 1 and board[x1][y1 - 1] == 0:
            next_.append(((x1, y1 - 1), (x2, y1)))
        if 0 <= x1 - 1 <= N - 1 and board[x1 - 1][y1] == 0 and board[x2 - 1][y2] == 0:
            next_.append(((x1 - 1, y1), (x2 - 1, y2)))
            next_.append(((x1 - 1, y1), (x1, y1)))
            next_.append(((x1 - 1, y2), (x2, y2)))
        if 0 <= x1 + 1 <= N - 1 and board[x1 + 1][y1] == 0 and board[x2 + 1][y2] == 0:
            next_.append(((x1 + 1, y1), (x2 + 1, y2)))
            next_.append(((x1, y1), (x1 + 1, y1)))
            next_.append(((x2, y2), (x1 + 1, y2)))
    else:
        if 0 <= x1 - 1 <= N - 1 and board[x1 - 1][y1] == 0:
            next_.append(((x1 - 1, y1), (x1, y1)))
        if 0 <= x2 + 1 <= N - 1 and board[x2 + 1][y2] == 0:
            next_.append(((x2, y2), (x2 + 1, y2)))
        if 0 <= y1 - 1 <= N - 1 and board[x1][y1 - 1] == 0 and board[x2][y2 - 1] == 0:
            next_.append(((x1, y1 - 1), (x2, y2 - 1)))
            next_.append(((x1, y1 - 1), (x1, y1)))
            next_.append(((x2, y2 - 1), (x2, y2)))
        if 0 <= y2 + 1 <= N - 1 and board[x1][y1 + 1] == 0 and board[x2][y2 + 1] == 0:
            next_.append(((x1, y1 + 1), (x2, y2 + 1)))
            next_.append(((x1, y1), (x1, y1 + 1)))
            next_.append(((x2, y2), (x2, y2 + 1)))
    return next_
def solution(board):
    n = len(board)
    visited = {((0, 0), (0, 1)) : 0}
    cur = ((0, 0), (0, 1))
    Q = deque()
    Q.append(cur)
    while Q:
        cur = Q.popleft()
        next_move = move(cur, board, n)
        for m in next_move:
            if (n-1,n-1) in m:
                return visited[cur] + 1
            if visited.setdefault(m, float('inf')) > visited[cur] + 1:
                visited[m] = visited[cur] + 1
                Q.append(m)

# BFS + DP 형태 풀이
# 처음엔 DP를 이차원 리스트로 구현하려했는데, 14점 맞고 포기 / 경로 이동에서 카운트가 제대로 안되는거 같은데 문제점을 모르겠음
# 다른 사람 풀이보다가 딕셔너리로 저장하길래 그렇게 해봤는데 답이 되더라
# 구성은 금방 생각했는데 구현이 매우 복잡하다
# 배운점
# 1. dict.setdefault(k, N)은 value가 존재하면 value, 없으면 N을 반환 / get도 똑같음

# 좋아요 가장 많은 풀이
from collections import deque

def solution(board):
    SIZE = len(board)
    ROW_WISE, COLUMN_WISE = range(2)
    OPEN, WALL = range(2)
    START = (0, 0, ROW_WISE)
    END_POINT = (SIZE-1, SIZE-1)
    DELTAS = ((0, 1), (1, 0), (0, -1), (-1, 0))
    queue = deque([START])
    visited = set()
    visited.add(START)
    moves_count = 0
    def _is_in_range(r, c):
        return 0 <= r < SIZE and 0 <= c < SIZE
    def _is_open(r, c):
        return board[r][c] == OPEN
    def _is_ok(r, c):
        return _is_in_range(r, c) and _is_open(r, c)
    def _yield_moves_rowwise(r, c):
        for dr, dc in DELTAS:
            new_r, new_c = r + dr, c + dc
            if _is_ok(new_r, new_c) and _is_ok(new_r, new_c+1):
                yield (new_r, new_c, ROW_WISE)
        if _is_ok(r+1, c) and _is_ok(r+1, c+1):
            yield (r, c, COLUMN_WISE)
            yield (r, c+1, COLUMN_WISE)
        if _is_ok(r-1, c) and _is_ok(r-1, c+1):
            yield (r-1, c, COLUMN_WISE)
            yield (r-1, c+1, COLUMN_WISE)
    def _yield_moves_columnwise(r, c):
        for dr, dc in DELTAS:
            new_r, new_c = r + dr, c + dc
            if _is_ok(new_r, new_c) and _is_ok(new_r+1, new_c):
                yield (new_r, new_c, COLUMN_WISE)
        if _is_ok(r, c-1) and _is_ok(r+1, c-1):
            yield (r, c-1, ROW_WISE)
            yield (r+1, c-1, ROW_WISE)
        if _is_ok(r, c+1) and _is_ok(r+1, c+1):
            yield (r, c, ROW_WISE)
            yield (r+1, c, ROW_WISE)
    while queue:
        moves_count += 1
        for _ in range(len(queue)):
            r, c, direction = queue.popleft()
            if direction == ROW_WISE:
                yield_func = _yield_moves_rowwise
            else:
                yield_func = _yield_moves_columnwise
            for new_coord in yield_func(r, c):
                if new_coord not in visited:
                    queue.append(new_coord)
                    visited.add(new_coord)
                    r, c, direction = new_coord
                    if (r, c+1) == END_POINT or (r+1, c) == END_POINT:
                        return moves_count

# 배운점
# 1. 함수에서 이터레이터 생성하는 yield
# https://dojang.io/mod/page/view.php?id=2412    /    https://tech.ssut.me/what-does-the-yield-keyword-do-in-python/

# 풍선 터트리기

In [ ]:
def solution(a):
    answer = 2
    if len(a) <= 2:
        return len(a)
    else:
        l, r = a[0], a[-1]
        for i in range(1, len(a)-1):
            if a[i] < l:
                l = a[i]
                answer += 1
            if a[-1-i] < r:
                r = a[-1-i]
                answer += 1
    return answer-1 if l==r else answer

# 이런 문제는 너무 어렵다 // 손으로 쓰면서 규칙찾느라 생고생함 ㅅㅂ
# 1. 양 끝단 수는 무조건 남길수 있음
# 2. 양 끝에서 출발해서 작은 수가 나오면 양 끝(l, r)을 갱신
# 3. l==r 이면 한개가 더 세지는거라 하나 빼고, 아니면 그대로 return
# len(a) == 3 여도 1 5 2 면 가운데 5를 못남김

# 좋아요 가장 많은 풀이
def solution(a):
    answer = 1
    M = min(a)
    for _ in range(2):
        m = a[0]
        i = 1
        while m != M:
            if m >= a[i]:
                m = a[i]
                answer += 1
            i += 1
        a.reverse()
    return answer

# 똑같은 풀이

